# Find cluster related terms using LIME

## Load the data

In [1]:
root_dir = '../..'
data_dir = 'data'
corpus_dir = 'corpus'
src_dir = 'src'

In [2]:
version = 'v2'

In [3]:
import os 
import sys

In [4]:
sys.path.append(os.path.join(root_dir, src_dir))

In [5]:
corpus_filename = f'wikidata_corpus_{version}.json'
corpus_filepath = os.path.join(root_dir, data_dir, corpus_dir, corpus_filename)

In [6]:
chunks_filename = f'wikidata_chunks_{version}.json'
chunks_filepath = os.path.join(root_dir, data_dir, corpus_dir, chunks_filename)

In [7]:
from training import TrainingCorpus

In [8]:
corpus = TrainingCorpus()
corpus.load(corpus_filepath)
corpus.load_chunks(chunks_filepath)

---

## Load the model

Check if GPU is available

In [9]:
import torch
print(torch.cuda.is_available())

True


In [10]:
from model import BertModel

Using TensorFlow backend.


In [11]:
model_dir = f'models/wikidata_bert_{version}'
model_dir_path = os.path.join(root_dir, data_dir, model_dir)

In [12]:
model = BertModel(model_dir_path, batch_size=128, use_cuda=True)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


---

## Find relevant terms for each cluster label using LIME

### Instantiate TermFinder

In [13]:
from termfinder import LimeTermFinder

In [14]:
term_finder = LimeTermFinder(model, corpus)

### Retrieve predicted labels for each instance in the corpus

In [15]:
import numpy as np

In [16]:
label_to_data_idx_dict = model.label_to_data_idx(corpus)

In [17]:
len(label_to_data_idx_dict)

195

### Retrieve relevant terms using LimeTermFinder

In [18]:
from tqdm.notebook import tqdm

In [19]:
df_data = []

In [20]:
for label_idx, data_idxs in tqdm(label_to_data_idx_dict.items()):
    for data_idx in tqdm(data_idxs, desc=f'Relevant terms for entity {label_idx}', leave=False):
        relevant_terms = term_finder.get_relevant_terms(data_idx, label_idx)

        if relevant_terms:
            for term, weight in relevant_terms.items():
                dict_entry = {'label': corpus.labels[label_idx], 
                              'term': term, 
                              'weight': weight, 
                              'data_id': corpus.docs[data_idx]}
                df_data.append(dict_entry)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Build a DataFrame out of `df_data`

In [21]:
df_data[:3]

[{'label': 'Q1754',
  'term': 'sweden',
  'weight': 0.42651232272320827,
  'data_id': 0},
 {'label': 'Q1754',
  'term': 'stockholm',
  'weight': 0.4007281724283034,
  'data_id': 0},
 {'label': 'Q1754',
  'term': 'city',
  'weight': 0.22206449681787813,
  'data_id': 0}]

In [22]:
import pandas as pd

In [23]:
relevant_terms_df = pd.DataFrame(df_data)

In [24]:
relevant_terms_df.head()

,label,term,weight,data_id
0,Q1754,sweden,0.426512,0
1,Q1754,stockholm,0.400728,0
2,Q1754,city,0.222064,0
3,Q1754,capital,0.218597,0
4,Q1754,capital,0.606604,1


---

## Save retrieved terms to a file

In [25]:
terms_dir = 'terms'
filename = f'relevant_terms_wikidata_bert_{version}.csv'
filepath = os.path.join(root_dir, data_dir, terms_dir, filename)

In [26]:
relevant_terms_df.to_csv(filepath, encoding='utf-8', index=False)

---